## 관찰 없이 추론하기
ReWOO 에서 Xu 등은 효과적인 도구 사용을 위해 다단계 플래너와 변수 대체를 결합한 에이전트를 제안. 다음과 같은 방식으로 ReACT 스타일 에이전트 아키텍처를 개선하도록 설계

1. 단일 패스에서 사용되는 전체 도구 체인을 생성하여 토큰 소비와 실행 시간을 줄인다. ( ReACT 스타일 에이전트 아키텍처는 중복된 접두사가 있는 많은 LLM 호출이 필요합니다(시스템 프롬프트와 이전 단계가 각 추론 단계에 대해 LLM에 제공되기 때문 ))
2. 미세 조정 프로세스를 간소화한다. 계획 데이터는 도구의 출력에 의존하지 않으므로 실제로 도구를 호출하지 않고도 모델을 미세 조정할 수 있다(이론적으로).

다음 다이어그램은 ReWOO의 전반적인 계산 그래프를 간략하게 설명.

![image](../image/4_Agent/ReWOO1.png)

ReWOO는 3개의 모듈로 구성되어 있습니다.

1. 🧠 플래너 : 다음 형식으로 계획을 생성.

``` bash
Plan: <reasoning>
#E1 = Tool[argument for tool]
Plan: <reasoning>
#E2 = Tool[argument for tool with #E1 variable substitution]
...
```

2. Worker : 제공된 인수로 도구를 실행.

3. 🧠 Solver : 도구 관찰을 기반으로 초기 작업에 대한 답을 생성.

🧠 이모지가 있는 모듈은 LLM 호출에 의존. 변수 대체를 사용하여 플래너 LLM에 대한 중복 호출을 피한다는 점에 유의.

In [1]:
import os
os.chdir("../../")
os.getcwd()

'/home/yeonwoo/study'

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [3]:
# 그래프 상태 정의
from typing import List
from typing_extensions import TypedDict


class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str

In [22]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-2024-08-06")

In [23]:
prompt = """다음 작업에서는 문제를 단계별로 해결할 수 있는 계획을 세우세요. 
각 계획에 대해 증거를 검색하기 위한 도구 입력과 함께 외부 도구를 표시합니다. 
나중에 도구에서 호출할 수 있는 변수 #E에 증거를 저장할 수 있습니다. (Plan, #E1, Plan, #E2, Plan, ...)

도구는 다음 중 하나일 수 있습니다:
(1) Google[input]: Google에서 결과를 검색하는 작업자입니다. 짧은 내용을 찾아야 할 때 유용합니다.
특정 주제에 대한 간결한 답변. 입력은 검색어여야 합니다.
(2) LLM[input]: 당신과 같은 사전 훈련된 LLM입니다. 일반적인 작업을 수행해야 할 때 유용합니다.
세계 지식과 상식. 문제 해결에 자신감이 있을 때 우선순위를 정하세요
당신 자신. 입력은 모든 명령어가 될 수 있습니다.

예를 들어,
Task: Thomas, Toby, Rebecca는 일주일에 총 157시간을 일했습니다. Thomas는 x시간 일했어요. Toby는 Thomas가 일한 것의 두 배보다 10시간 적게 일했고, Rebecca는 Toby보다 8시간 적게 일했습니다. Rebecca는 몇 시간 일했습니까?
Plan: Thomas가 x시간 일했다면 문제를 대수식으로 번역하고 풀어보세요.
with Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x - 10) + ((2x - 10) - 8) = 157]
Plan: 토마스가 몇 시간 일했는지 알아보세요. #E2 = LLM[What is x, given #E1]
Plan: 레베카가 일한 시간을 계산하세요. #E3 = Calculator[(2 * #E2 - 10) - 8]

Begin! 
풍부한 세부정보로 계획을 설명하세요. 각 계획 뒤에는 하나의 #E만 와야 합니다.

Task: {task}"""

In [35]:
task = "2024년 리그오브레전드의 우승팀인 T1의 미드라이너의 정확한 고향은 어디인가?"

In [36]:
result = llm.invoke(prompt.format(task=task))

In [37]:
print(result.content)

Plan: 먼저, T1 팀의 2024년 리그 오브 레전드 우승 여부를 확인합니다. 이는 T1 팀이 실제로 우승했는지 확인하기 위한 첫 번째 단계입니다. #E1 = Google["2024 리그 오브 레전드 우승팀 T1"]

Plan: T1의 미드라이너가 누구인지 확인합니다. T1 팀의 미드라이너를 알아야 그의 고향을 찾을 수 있습니다. #E2 = Google["T1 리그 오브 레전드 2024 미드라이너"]

Plan: T1 미드라이너의 고향을 확인합니다. 이전 단계에서 찾은 미드라이너의 이름을 사용하여 그 선수의 고향을 검색합니다. #E3 = Google["[이름] 고향"]


In [38]:
# planner note
import re

from langchain_core.prompts import ChatPromptTemplate

# Regex to match expressions of the form E#... = ...[...]
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
planner = prompt_template | llm


def get_plan(state: ReWOO):
    task = state["task"]
    result = planner.invoke({"task": task})
    # Find all matches in the sample text
    matches = re.findall(regex_pattern, result.content)
    return {"steps": matches, "plan_string": result.content}

In [39]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [40]:
def _get_current_task(state: ReWOO):
    if "results" not in state or state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1


def tool_execution(state: ReWOO):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1]
    _results = (state["results"] or {}) if "results" in state else {}
    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)
    if tool == "Google":
        result = search.invoke(tool_input)
    elif tool == "LLM":
        result = llm.invoke(tool_input)
    else:
        raise ValueError
    _results[step_name] = str(result)
    return {"results": _results}

In [41]:
solve_prompt = """다음 작업이나 문제를 해결하세요. 
문제를 해결하기 위해 단계별 계획을 수립하고 각 계획에 해당하는 증거를 검색했습니다.
긴 증거에는 관련 없는 정보가 포함될 수 있으므로 주의해서 사용하세요.

{plan}

이제 위에 제공된 증거에 따라 질문이나 작업을 해결하십시오. 추가 단어 없이 직접 답변해 보세요.

Task: {task}
Response:"""


def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = (state["results"] or {}) if "results" in state else {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {tool}[{tool_input}]"
    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = llm.invoke(prompt)
    return {"result": result.content}

In [42]:
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        # We have executed all tasks
        return "solve"
    else:
        # We are still executing tasks, loop back to the "tool" node
        return "tool"

In [43]:
from langgraph.graph import END, StateGraph, START

graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)
graph.add_conditional_edges("tool", _route)
graph.add_edge(START, "plan")

app = graph.compile()

In [44]:
for s in app.stream({"task": task}):
    print(s)
    print("---")

{'plan': {'plan_string': 'Plan: 2024년 리그오브레전드 우승팀인 T1의 미드라이너가 누구인지 확인합니다. #E1 = Google["2024 League of Legends T1 mid laner"]\n\nPlan: #E1에서 얻은 정보에 따라 해당 선수의 고향을 찾습니다. #E2 = Google["[Player Name] hometown"]', 'steps': [('2024년 리그오브레전드 우승팀인 T1의 미드라이너가 누구인지 확인합니다. ', '#E1', 'Google', '"2024 League of Legends T1 mid laner"'), ('#E1에서 얻은 정보에 따라 해당 선수의 고향을 찾습니다. ', '#E2', 'Google', '"[Player Name')]}}
---
{'tool': {'results': {'#E1': '[{\'url\': \'https://www.reddit.com/r/SKTT1/comments/1grqx2z/t1_roster_megathread/\', \'content\': \'Nov 15, 2024 · T1 League of Legends ; Faker, Lee Sang-hyeok (이상혁), Mid Laner ; Gumayusi, Lee Min-hyeong (이민형), Bot Laner ; Keria, Ryu Min-seok (류민석)\\xa0...T1 vs. HLE / LCK 2024 Summer - Week 5 / Post Match DiscussionT1 Faker is the 2nd greatest mid-laner : r/leagueoflegends - RedditEvery Mid Laner tournament wins after Worlds 2024. The faded ...All trophies won by each mid laner, after Worlds 2024 - RedditMore results from www.reddit.com\'}, {\'url\': \'https:

In [45]:
print(s["solve"]["result"])

이상혁 (Faker)은 대한민국 서울 출신입니다.
